### Spectra

In [ ]:
import os
import re


'''We create a list with the files names with form *.txt in the folder data_34000_19032025.
'''
folder_path = 'TABLES/mytd_gaussian/'

# List all .txt files in the folder, 
#endswith: this fuction to verifica if a str finishes in a subfix
all_filenames = [f for f in os.listdir(folder_path) if f.endswith(".txt")]

dfs = []  # lista temporal

# Create master DataFrame
for i in all_filenames:
    df = pd.read_csv('TABLES/mytd_gaussian/'+str(i), names=['energy', 'counts'], header=None, sep="\s+")
    df_T = pd.DataFrame(df['counts']).T
    
    # Expresión regular para encontrar el número después de 'ID'
    match = re.search(r'ID-(\d+)', str(i))
    if match:
        id_source = match.group(1)
        
    df_T['id_source'] = id_source
    
    dfs.append(df_T)

# counts + energy
df_master = pd.concat(dfs, ignore_index=True)

print(df_master.shape)

(34000, 4097)


In [4]:
'''The first row is the energy.
'''

df_master.tail(3)

,0,1,2,3,4,5,6,7,8,9,...,4087,4088,4089,4090,4091,4092,4093,4094,4095,id_source
33997,0.0,0.0,0.0,0.0,0.0,24.999948,0.000000,100.000099,149.999695,100.000099,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16401
33998,0.0,0.0,0.0,0.0,0.0,0.000000,25.000025,25.000025,99.999794,50.000050,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16873
33999,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,25.000025,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21448


### Physical parameters + exp_time and PL

In [5]:
Target = pd.read_csv(path_data+"parameter_gaussian_grid_all_34_BetterPresicion_Z_Nhg.txt", sep="\t")
Target.rename(columns={Target.columns[0]: 'id_source'}, inplace=True)

### Concat

In [6]:
df_master['id_source'] = df_master['id_source'].astype(int)

df_master = df_master.sort_values(by='id_source')
Target = Target.sort_values(by='id_source')

id_source = df_master.id_source.values

df_master = df_master.set_index('id_source')
Target = Target.set_index('id_source')

print(set(id_source) - set(df_master.index))
print(set(id_source) - set(Target.index))

df_master[Target.columns] = 0
df_master.loc[id_source, Target.columns] = Target.loc[id_source, Target.columns]
display(df_master.shape)

set()
set()


/tmp/ipykernel_341672/238049487.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.874 0.808 0.771 ... 0.671 0.735 0.799]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_master.loc[id_source, Target.columns] = Target.loc[id_source, Target.columns]
/tmp/ipykernel_341672/238049487.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.663 0.696 0.685 ... 0.237 0.182 0.33 ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_master.loc[id_source, Target.columns] = Target.loc[id_source, Target.columns]
/tmp/ipykernel_341672/238049487.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1.57 1.52 1.49 ... 1.61 1.69 1.59]' has dtype incompatible with int64

(34000, 4104)

### Split

In [7]:
df_master = df_master.reset_index()

In [8]:
PP = ['N_H_Z','N_H_S','Gamma','A_S']

X = df_master.drop(columns=PP).copy()
y = df_master[PP+['id_source']]

print('X:', X.shape, 'y:', y.shape)

# Split the total data in 70% training set and 30% test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# Split the total data in 80% training set and 20% test set
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print(f'Tamaño del conjunto de entrenamiento: {X_train.shape}')
print(f'Size of the validation set: {X_val.shape}')
print(f'Tamaño del conjunto de prueba: {X_test.shape}')


X: (34000, 4101) y: (34000, 5)
Tamaño del conjunto de entrenamiento: (19040, 4101)
Size of the validation set: (4760, 4101)
Tamaño del conjunto de prueba: (10200, 4101)


**Two approaches:**

    1) Features: counts in 2 to 50 KeV
    2) Features: counts in 2 to 50 KeV + exp_time and PL_norm
    3) Featres: counts in 3 to 30 KeV
    4) Featres: counts in 3 to 30 KeV  + exp_time and PL_norm
    5) Featres: counts in 3 to 30 KeV  + exp_time
    6) Features: counts in 3 to 30 KeV + exp_time + z + NH_Gal

In [9]:
energy = pd.read_csv('TABLES/mytd_gaussian/'+all_filenames[1], names=['energy', 'counts'], header=None, sep="\s+")
energy = pd.DataFrame(energy['energy']).T

In [10]:
energy

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
energy,1.62,1.66,1.7,1.74,1.78,1.82,1.86,1.9,1.94,1.98,...,165.059998,165.100006,165.139999,165.179993,165.220001,165.26001,165.300003,165.339996,165.380005,165.419998


**1) Features: counts**

**2) Features: counts + exp_time and PL**

**3) Features: counts 3 to 30 KeV**

**4) Features: counts 3 to 30 KeV + EXP TIME AND PLL_NORM**

**5) Features: counts 3 to 30 KeV + EXP TIME**

**6) Features: counts in 3 to 30 KeV + exp_time + z + NH_Gal**

In [11]:
app_4 = energy.loc[:, (energy.iloc[0]>=3) & (energy.iloc[0]<30)]
counts = app_4.columns.values

features = list(counts) +['DOUBLE_EXP_TIME_(s)', 'z', 'nH_G', 'id_source']

DFX_train = pd.DataFrame(X_train)[features]  
DFX_val = pd.DataFrame(X_val)[features]
DFX_test = pd.DataFrame(X_test)[features]

path_app1 = path_data + 'APP_6_mytd_gau/'
DFX_train.to_csv(path_app1 + "X_train.csv", index=False)
DFX_val.to_csv(path_app1 + "X_val.csv", index=False)
DFX_test.to_csv(path_app1 + "X_test.csv", index=False)

y_train.to_csv(path_app1+"y_train.csv", index=False)
y_val.to_csv(path_app1+"y_val.csv", index=False)
y_test.to_csv(path_app1+"y_test.csv", index=False)

In [12]:
display(DFX_train.shape)
display(DFX_val.shape)
display(DFX_test.shape)

(19040, 679)

(4760, 679)

(10200, 679)